# Instalando bibliotecas necessárias

In [ ]:
%pip install beautifulsoup4 pandas requests

# Importando bibliotecas

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time

# Funções de extração das partidas e status dos players e champion

In [17]:
def extract_match_data(urls: list[str]):
    patches = set()
    data_list = []

    for url in urls:
        print(url)
        time.sleep(3)
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            body = soup.find('body')
            table_content = body.find('div', class_='wide-content-scroll').find('table')
            tbody = table_content.find('tbody')

            for tr in tbody.findAll('tr'):
                data = tr.find_all('td')

                if len(data) > 0:
                    date = data[0].get_text()
                    patch = data[1].find('a').get_text() if data[1].find('a') else None

                    patches.add(patch)

                    blue_side = data[2].find('a').attrs['title']
                    red_side = data[3].find('a').attrs['title']
                    winner = data[4].find('a').attrs['title']
                    blue_bans = [span.attrs['title'] for span in data[5].find_all('span')]
                    red_bans = [span.attrs['title'] for span in data[6].find_all('span')]
                    blue_picks = [span.attrs['title'] for span in data[7].find_all('span')]
                    red_picks = [span.attrs['title'] for span in data[8].find_all('span')]
                    blue_roster = [a.attrs['title'] for a in data[9].find_all('a')]
                    red_roster = [a.attrs['title'] for a in data[10].find_all('a')]
                    
                    championship_id = url.replace('https://lol.fandom.com/wiki/', '').split('/')

                    data_list.append({
                        'championship_id': f"{championship_id[0]}/{championship_id[1]}/{championship_id[2]}",
                        'date': date,
                        'patch': patch,
                        'blue_side': blue_side,
                        'red_side': red_side,
                        'winner': winner,
                        'blue_bans': blue_bans,
                        'red_bans': red_bans,
                        'blue_picks': blue_picks,
                        'red_picks': red_picks,
                        'blue_roster': blue_roster,
                        'red_roster': red_roster
                    })
    
    return data_list

In [18]:
def extract_champion_statistics_data(urls: list[str]):
    champion_stats_data = {}
    for url in urls:
        print(url)
        time.sleep(3)
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            body = soup.find('body')
            table_content = body.find('div', class_='wide-content-scroll').find('table')
            tbody = table_content.find('tbody')
            
            championship_id = url.replace('https://lol.fandom.com/wiki/', '').split('/')
            champion_stats_data[f"{championship_id[0]}/{championship_id[1]}/{championship_id[2]}"] = []
            for tr in tbody.find_all('tr'):
                data = tr.find_all('td')

                if len(data) > 0:
                    if data[0].get_text() != 'Top':
                        champion = data[0].get_text()
                        games = data[1].get_text()
                        pb = data[2].get_text()
                        b = data[3].get_text()
                        g = data[4].get_text()
                        played_by_number_players = data[5].get_text()
                        win = data[6].get_text()
                        lose = data[7].get_text()
                        winrate = data[8].get_text()
                        kill = data[9].get_text()
                        death = data[10].get_text()
                        assist = data[11].get_text()
                        kda = data[12].get_text()
                        cs = data[13].get_text()
                        csm = data[14].get_text()
                        gold = data[15].get_text()
                        gold_per_m = data[16].get_text()
                        dmg = data[17].get_text()
                        dmg_per_m = data[18].get_text()
                        kpar = data[19].get_text()
                        ks = data[20].get_text()
                        gs = data[21].get_text()
                        As = [span.attrs['title'] for span in data[22].find_all('span')]

                        champion_stats_data[f"{championship_id[0]}/{championship_id[1]}/{championship_id[2]}"].append({
                            'champion': champion,
                            'games': games,
                            'pb': pb,
                            'b': b,
                            'g': g,
                            'played_by_number_players': played_by_number_players,
                            'win': win,
                            'lose': lose,
                            'winrate': winrate,
                            'kill': kill,
                            'death': death,
                            'assist': assist,
                            'kda': kda,
                            'cs': cs,
                            'csm': csm,
                            'gold': gold,
                            'gold_per_m': gold_per_m,
                            'dmg': dmg,
                            'dmg_per_m': dmg_per_m,
                            'kpar': kpar,
                            'ks': ks,
                            'gs': gs,
                            'As': As
                        })
    return champion_stats_data

In [19]:
def extract_player_statistics_data(urls: list[str]):
    player_stats_data = {}
    for url in urls:
        print(url)
        time.sleep(3)
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            body = soup.find('body')
            table_content = body.find('div', class_='wide-content-scroll').find('table')
            tbody = table_content.find('tbody')
            
            championship_id = url.replace('https://lol.fandom.com/wiki/', '').split('/')
            player_stats_data[f"{championship_id[0]}/{championship_id[1]}/{championship_id[2]}"] = []
            for tr in tbody.find_all('tr'):
                data = tr.find_all('td')

                if len(data) > 0:
                    if data[0].get_text() != 'Top':
                        team = data[0].find('a').attrs['data-to-id'].replace('_', ' ')
                        player = data[1].get_text()
                        games = data[2].get_text()
                        w = data[3].get_text()
                        l = data[4].get_text()
                        wr = data[5].get_text()
                        k = data[6].get_text()
                        d = data[7].get_text()
                        a = data[8].get_text()
                        kda = data[9].get_text()
                        cs = data[10].get_text()
                        csm = data[11].get_text()
                        gold = data[12].get_text()
                        gm = data[13].get_text()
                        dmg = data[14].get_text()
                        dmgm = data[15].get_text()
                        kpar = data[16].get_text()
                        ks = data[17].get_text()
                        gs = data[18].get_text()
                        cp = data[19].get_text()
                        champs = [span.attrs['title'] for span in data[20].find_all('span')]

                        player_stats_data[f"{championship_id[0]}/{championship_id[1]}/{championship_id[2]}"].append({
                            'team': team,
                            'player': player,
                            'games': games,
                            'w': w,
                            'l': l,
                            'wr': wr,
                            'k': k,
                            'd': d,
                            'a': a,
                            'kda': kda,
                            'cs': cs,
                            'csm': csm,
                            'gold': gold,
                            'gm': gm,
                            'dmg': dmg,
                            'dmgm': dmgm,
                            'kpar': kpar,
                            'ks': ks,
                            'gs': gs,
                            'cp': cp,
                            'champs': champs
                        })
    return player_stats_data

In [6]:
def save_list_into_datasets(data_list, name_df):
    if not os.path.exists('../datasets'):
        os.mkdir('../datasets')
    df = pd.DataFrame(data_list)
    df.to_csv(f'../datasets/{name_df}.csv', index=False, encoding='utf-8', sep=';')

# WebScraping

## Worlds

In [ ]:
match_history_worlds = extract_match_data(
    [
        # play in worlds: 2017 to 2024
        'https://lol.fandom.com/wiki/2017_Season_World_Championship/Play-In/Match_History',
        'https://lol.fandom.com/wiki/2018_Season_World_Championship/Play-In/Match_History',
        'https://lol.fandom.com/wiki/2019_Season_World_Championship/Play-In/Match_History',
        'https://lol.fandom.com/wiki/2020_Season_World_Championship/Play-In/Match_History',
        'https://lol.fandom.com/wiki/2021_Season_World_Championship/Play-In/Match_History',
        'https://lol.fandom.com/wiki/2022_Season_World_Championship/Play-In/Match_History',
        'https://lol.fandom.com/wiki/2023_Season_World_Championship/Play-In/Match_History',
        'https://lol.fandom.com/wiki/2024_Season_World_Championship/Play-In/Match_History',

        # main event worlds: 2017 to 2024
        'https://lol.fandom.com/wiki/2017_Season_World_Championship/Main_Event/Match_History',
        'https://lol.fandom.com/wiki/2018_Season_World_Championship/Main_Event/Match_History',
        'https://lol.fandom.com/wiki/2019_Season_World_Championship/Main_Event/Match_History',
        'https://lol.fandom.com/wiki/2020_Season_World_Championship/Main_Event/Match_History',
        'https://lol.fandom.com/wiki/2021_Season_World_Championship/Main_Event/Match_History',
        'https://lol.fandom.com/wiki/2022_Season_World_Championship/Main_Event/Match_History',
        'https://lol.fandom.com/wiki/2023_Season_World_Championship/Main_Event/Match_History',
        'https://lol.fandom.com/wiki/2024_Season_World_Championship/Main_Event/Match_History',
    ]
)

save_list_into_datasets(match_history_worlds, 'match_history_worlds')

In [ ]:
champion_statistics_worlds = extract_champion_statistics_data(
    [
        # play in worlds: 2017 to 2024
        'https://lol.fandom.com/wiki/2017_Season_World_Championship/Play-In/Champion_Statistics',
        'https://lol.fandom.com/wiki/2018_Season_World_Championship/Play-In/Champion_Statistics',
        'https://lol.fandom.com/wiki/2019_Season_World_Championship/Play-In/Champion_Statistics',
        'https://lol.fandom.com/wiki/2020_Season_World_Championship/Play-In/Champion_Statistics',
        'https://lol.fandom.com/wiki/2021_Season_World_Championship/Play-In/Champion_Statistics',
        'https://lol.fandom.com/wiki/2022_Season_World_Championship/Play-In/Champion_Statistics',
        'https://lol.fandom.com/wiki/2023_Season_World_Championship/Play-In/Champion_Statistics',
        'https://lol.fandom.com/wiki/2024_Season_World_Championship/Play-In/Champion_Statistics',

        # main event worlds: 2017 to 2024
        'https://lol.fandom.com/wiki/2017_Season_World_Championship/Main_Event/Champion_Statistics',
        'https://lol.fandom.com/wiki/2018_Season_World_Championship/Main_Event/Champion_Statistics',
        'https://lol.fandom.com/wiki/2019_Season_World_Championship/Main_Event/Champion_Statistics',
        'https://lol.fandom.com/wiki/2020_Season_World_Championship/Main_Event/Champion_Statistics',
        'https://lol.fandom.com/wiki/2021_Season_World_Championship/Main_Event/Champion_Statistics',
        'https://lol.fandom.com/wiki/2022_Season_World_Championship/Main_Event/Champion_Statistics',
        'https://lol.fandom.com/wiki/2023_Season_World_Championship/Main_Event/Champion_Statistics',
        'https://lol.fandom.com/wiki/2024_Season_World_Championship/Main_Event/Champion_Statistics',
    ]
)

champion_statistics_worlds_transformed = []
for championship_id, all_data in champion_statistics_worlds.items():
    for data in all_data:
        champion_statistics_worlds_transformed.append({'championship_id': championship_id, **data})

save_list_into_datasets(champion_statistics_worlds_transformed, 'champion_statistics_worlds')

In [ ]:
player_statistics_worlds = extract_player_statistics_data(
    [
        # play in worlds: 2017 to 2024
        'https://lol.fandom.com/wiki/2017_Season_World_Championship/Play-In/Player_Statistics',
        'https://lol.fandom.com/wiki/2018_Season_World_Championship/Play-In/Player_Statistics',
        'https://lol.fandom.com/wiki/2019_Season_World_Championship/Play-In/Player_Statistics',
        'https://lol.fandom.com/wiki/2020_Season_World_Championship/Play-In/Player_Statistics',
        'https://lol.fandom.com/wiki/2021_Season_World_Championship/Play-In/Player_Statistics',
        'https://lol.fandom.com/wiki/2022_Season_World_Championship/Play-In/Player_Statistics',
        'https://lol.fandom.com/wiki/2023_Season_World_Championship/Play-In/Player_Statistics',
        'https://lol.fandom.com/wiki/2024_Season_World_Championship/Play-In/Player_Statistics',

        # main event worlds: 2017 to 2024
        'https://lol.fandom.com/wiki/2017_Season_World_Championship/Main_Event/Player_Statistics',
        'https://lol.fandom.com/wiki/2018_Season_World_Championship/Main_Event/Player_Statistics',
        'https://lol.fandom.com/wiki/2019_Season_World_Championship/Main_Event/Player_Statistics',
        'https://lol.fandom.com/wiki/2020_Season_World_Championship/Main_Event/Player_Statistics',
        'https://lol.fandom.com/wiki/2021_Season_World_Championship/Main_Event/Player_Statistics',
        'https://lol.fandom.com/wiki/2022_Season_World_Championship/Main_Event/Player_Statistics',
        'https://lol.fandom.com/wiki/2023_Season_World_Championship/Main_Event/Player_Statistics',
        'https://lol.fandom.com/wiki/2024_Season_World_Championship/Main_Event/Player_Statistics',
    ]
)

player_statistics_worlds_transformed = []
for championship_id, all_data in player_statistics_worlds.items():
    for data in all_data:
        player_statistics_worlds_transformed.append({'championship_id': championship_id, **data})

save_list_into_datasets(player_statistics_worlds_transformed, 'player_statistics_worlds')

## CBLoL

In [ ]:
match_history_cblol = extract_match_data(
    [
        # cblol - matches split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_1/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_2/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_2/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_2/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_2/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_2/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_2/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_2/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_2/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_2/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_2/Match_History',

        # cblol - playoffs split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_1_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_2_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_2_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_2_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_2_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_2_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_2_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_2_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_2_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_2_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_2_Playoffs/Match_History'
    ]
)

save_list_into_datasets(match_history_cblol, 'match_history_cblol')

In [ ]:
champion_statistics_cblol = extract_champion_statistics_data(
    [
        # cblol - matches split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_1/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_2/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_2/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_2/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_2/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_2/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_2/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_2/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_2/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_2/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_2/Champion_Statistics',

        # cblol - playoffs split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_1_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_2_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_2_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_2_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_2_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_2_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_2_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_2_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_2_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_2_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_2_Playoffs/Champion_Statistics'
    ]
)

champion_statistics_cblol_transformed = []
for championship_id, all_data in champion_statistics_cblol.items():
    for data in all_data:
        champion_statistics_cblol_transformed.append({'championship_id': championship_id, **data})

save_list_into_datasets(champion_statistics_cblol_transformed, 'champion_statistics_cblol')

In [ ]:
player_statistics_cblol = extract_player_statistics_data(
    [
        # cblol - matches split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_1/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_2/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_2/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_2/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_2/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_2/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_2/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_2/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_2/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_2/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_2/Player_Statistics',

        # cblol - playoffs split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_1_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2015_Season/Split_2_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2016_Season/Split_2_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2017_Season/Split_2_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2018_Season/Split_2_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2019_Season/Split_2_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2020_Season/Split_2_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2021_Season/Split_2_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2022_Season/Split_2_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_2_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_2_Playoffs/Player_Statistics'
    ]
)

player_statistics_cblol_transformed = []
for championship_id, all_data in player_statistics_cblol.items():
    for data in all_data:
        player_statistics_cblol_transformed.append({'championship_id': championship_id, **data})

save_list_into_datasets(player_statistics_cblol_transformed, 'player_statistics_cblol')

## LPL

In [ ]:
match_history_lpl = extract_match_data(
    [
        # lpl - matches split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LPL/2015_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2015_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Summer_Season/Match_History',

        # lpl - playoffs split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LPL/2015_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2015_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Summer_Playoffs/Match_History'
    ]
)

save_list_into_datasets(match_history_lpl, 'match_history_lpl')

In [ ]:
champion_statistics_lpl = extract_champion_statistics_data(
    [
        # lpl - matches split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LPL/2015_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2015_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Summer_Season/Champion_Statistics',

        # lpl - playoffs split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LPL/2015_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2015_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Summer_Playoffs/Champion_Statistics'
    ]
)

champion_statistics_lpl_transformed = []
for championship_id, all_data in champion_statistics_lpl.items():
    for data in all_data:
        champion_statistics_lpl_transformed.append({'championship_id': championship_id, **data})

save_list_into_datasets(champion_statistics_lpl_transformed, 'champion_statistics_lpl')

In [ ]:
player_statistics_lpl = extract_player_statistics_data(
    [
        # lpl - matches split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LPL/2015_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2015_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Summer_Season/Player_Statistics',

        # lpl - playoffs split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LPL/2015_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2015_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2016_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2017_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2018_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2019_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2020_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2021_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2022_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2023_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LPL/2024_Season/Summer_Playoffs/Player_Statistics'
    ]
)

player_statistics_lpl_transformed = []
for championship_id, all_data in player_statistics_lpl.items():
    for data in all_data:
        player_statistics_lpl_transformed.append({'championship_id': championship_id, **data})

save_list_into_datasets(player_statistics_lpl_transformed, 'player_statistics_lpl')

## LCK

In [ ]:
match_history_lck = extract_match_data(
    [
        # lck - matches split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LCK/2015_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Spring_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2015_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Summer_Season/Match_History',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Summer_Season/Match_History',

        # lck - playoffs split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LCK/2015_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Spring_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2015_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Summer_Playoffs/Match_History',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Summer_Playoffs/Match_History'
    ]
)

save_list_into_datasets(match_history_lck, 'match_history_lck')

In [ ]:
champion_statistics_lck = extract_champion_statistics_data(
    [
        # lck - matches split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LCK/2015_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Spring_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2015_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Summer_Season/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Summer_Season/Champion_Statistics',

        # lck - playoffs split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LCK/2015_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Spring_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2015_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Summer_Playoffs/Champion_Statistics',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Summer_Playoffs/Champion_Statistics'
    ]
)

champion_statistics_lck_transformed = []
for championship_id, all_data in champion_statistics_lck.items():
    for data in all_data:
        champion_statistics_lck_transformed.append({'championship_id': championship_id, **data})

save_list_into_datasets(champion_statistics_lck_transformed, 'champion_statistics_lck')

In [ ]:
player_statistics_lck = extract_player_statistics_data(
    [
        # lck - matches split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LCK/2015_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Spring_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2015_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Summer_Season/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Summer_Season/Player_Statistics',

        # lck - playoffs split 1 and split 2: 2015 to 2024
        'https://lol.fandom.com/wiki/LCK/2015_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Spring_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2015_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2016_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2017_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2018_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2019_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2020_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2021_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2022_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2023_Season/Summer_Playoffs/Player_Statistics',
        'https://lol.fandom.com/wiki/LCK/2024_Season/Summer_Playoffs/Player_Statistics'
    ]
)

player_statistics_lck_transformed = []
for championship_id, all_data in player_statistics_lck.items():
    for data in all_data:
        player_statistics_lck_transformed.append({'championship_id': championship_id, **data})

save_list_into_datasets(player_statistics_lck_transformed, 'player_statistics_lck')

## LCS